In [3]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time

pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
#used to play sounds
mixer.init()
sound = mixer.Sound(r'G:\Drowsiness detection\alarm.wav')

In [5]:
# This code loads the Haar cascade classifiers for face, left eye, and right eye detection.
face = cv2.CascadeClassifier(r'G:\Drowsiness detection\haar cascade files\haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier(r'G:\Drowsiness detection\haar cascade files\haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier(r'G:\Drowsiness detection\haar cascade files\haarcascade_righteye_2splits.xml')

In [7]:
lbl=['Close','Open']

model = load_model(r'C:\Users\rjhar\models\Drver Drowsiness  Detection CNN_64_2.h6') # Load the CNN model
path = os.getcwd() # Get the current working directory
cap = cv2.VideoCapture(0) # Initialize the video capture object
font = cv2.FONT_HERSHEY_COMPLEX_SMALL # Set the font for the text
# Initialize the score and counter variables
count=0  
score=0
thicc=2
rpred=[99]  #Taking Value Array of Integers
lpred=[99]

while(True):
    ret, frame = cap.read() #Boolean value that indicates whether or not a frame was successfully read from the webcam.
    height,width = frame.shape[:2] #Extract the first two elements from the frame.shape tuple

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert the frame to grayscale
    
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))  # Detect faces in the frame
    left_eye = leye.detectMultiScale(gray)  # Detect the eyes in the frame
    right_eye =  reye.detectMultiScale(gray)

    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )  # Draw a rectangle around the face in the frame

    for (x,y,w,h) in faces:     # Loop over the faces in the frame
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 ) # Draw a rectangle around the face

    for (x,y,w,h) in right_eye:    # Get the eye region from the frame
        r_eye=frame[y:y+h,x:x+w]
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)   # Convert the eye region to grayscale
        r_eye = cv2.resize(r_eye,(64,64))     # Resize the eye region to 64x64 pixels
        r_eye= r_eye/255     # Normalize the eye region
        r_eye=  r_eye.reshape(64,64,-1)   # Reshape the eye region to a 4D tensor
        r_eye = np.expand_dims(r_eye,axis=0)       # Expand the dimensions of the eye region
        rpred = np.argmax(model.predict(r_eye), axis=-1)  # Predict the eye state
        if(rpred[0]==1):
            lbl='Open' 
        if(rpred[0]==0):
            lbl='Closed'
        break

    for (x,y,w,h) in left_eye:
        l_eye=frame[y:y+h,x:x+w]
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)    # Convert the eye region to grayscale
        l_eye = cv2.resize(l_eye,(64,64))# Resize the eye region to 64x64 pixels
        l_eye= l_eye/255
        l_eye=l_eye.reshape(64,64,-1)   # Reshape the eye region to a 4D tensor
        l_eye = np.expand_dims(l_eye,axis=0)     # Expand the dimensions of the eye region
        lpred = np.argmax(model.predict(l_eye), axis=-1)  # Predict the eye state
        if(lpred[0]==1):
            lbl='Open'   
        if(lpred[0]==0):
            lbl='Closed'
        break
# If both eyes are closed, increase the score
    if(rpred[0]==0 and lpred[0]==0):
        score=score+1
        cv2.putText(frame,"Closed",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    # Otherwise, decrease the score
    else:
        score=score-1
        cv2.putText(frame,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
        # If the score is less than 0, reset it to 0
    if(score<0):
        score=0   
    cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    # If the score is greater than 5, beep the alarm and increase the thickness of the rectangle 
    if(score>5):  
        cv2.putText(frame, "Alert!", (10, height-40), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1, cv2.LINE_AA)
        #person is feeling sleepy so we beep the alarm and  Save the frame as an image
       # cv2.imwrite(os.path.join('G:\Drowsiness detection\Results','image.jpg'),frame)
   
        try:
            sound.play() # Try to play the alarm sound
            
        except:  # isplaying = False
            pass
        if(thicc<16):    # Increase the thickness of the rectangle
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc)  # Draw a rectangle around the frame
    cv2.imshow('frame',frame)
    #cv2.imwrite(os.path.join('G:\Drowsiness detection\Results','image.jpg'),frame)
    screenshot = frame.copy()

    # Get the current time
    time_now = time.strftime("%Y-%m-%d_%H-%M-%S")

    # Save the screenshot with the current time as the image name
    cv2.imwrite(os.path.join('G:\Drowsiness detection\Results', f'image_{time_now}.jpg'), screenshot)
    if cv2.waitKey(1) & 0xFF == ord('q'):    # If the user presses the 'q' key, break the loop
        break
cap.release()   # Release the video capture object
cv2.destroyAllWindows()


1/1 [==============================] - 0s 24ms/step
